In [1]:
#imports
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
#Scrape the single game shots
base_url = 'https://understat.com/match/'
match = str(input('Please enter match id:'))
url = base_url + match

Please enter match id:14755


In [3]:
url

'https://understat.com/match/14755'

In [4]:
res = requests.get(url) #request to the match url
soup = BeautifulSoup(res.content, 'lxml')
scripts = soup.find_all('script') # get only the 'script' tags

In [5]:
scripts

[<script>
 			var THEME = localStorage.getItem("theme") || 'DARK';
 			document.body.className = "theme-" + THEME.toLowerCase();
 		</script>,
 <script>
 	var shotsData 	= JSON.parse('\x7B\x22h\x22\x3A\x5B\x7B\x22id\x22\x3A\x22416977\x22,\x22minute\x22\x3A\x221\x22,\x22result\x22\x3A\x22BlockedShot\x22,\x22X\x22\x3A\x220.785\x22,\x22Y\x22\x3A\x220.7180000305175781\x22,\x22xG\x22\x3A\x220.022559041157364845\x22,\x22player\x22\x3A\x22Eddie\x20Nketiah\x22,\x22h_a\x22\x3A\x22h\x22,\x22player_id\x22\x3A\x226482\x22,\x22situation\x22\x3A\x22OpenPlay\x22,\x22season\x22\x3A\x222020\x22,\x22shotType\x22\x3A\x22RightFoot\x22,\x22match_id\x22\x3A\x2214755\x22,\x22h_team\x22\x3A\x22Arsenal\x22,\x22a_team\x22\x3A\x22Everton\x22,\x22h_goals\x22\x3A\x220\x22,\x22a_goals\x22\x3A\x221\x22,\x22date\x22\x3A\x222021\x2D04\x2D23\x2019\x3A00\x3A00\x22,\x22player_assisted\x22\x3A\x22Thomas\x20Partey\x22,\x22lastAction\x22\x3A\x22Pass\x22\x7D,\x7B\x22id\x22\x3A\x22416978\x22,\x22minute\x22\x3A\x221\x22,\x22re

In [6]:
#get only the shotsData, usually 2nd in the list
strings = scripts[1].string

In [7]:
strings

"\n\tvar shotsData \t= JSON.parse('\\x7B\\x22h\\x22\\x3A\\x5B\\x7B\\x22id\\x22\\x3A\\x22416977\\x22,\\x22minute\\x22\\x3A\\x221\\x22,\\x22result\\x22\\x3A\\x22BlockedShot\\x22,\\x22X\\x22\\x3A\\x220.785\\x22,\\x22Y\\x22\\x3A\\x220.7180000305175781\\x22,\\x22xG\\x22\\x3A\\x220.022559041157364845\\x22,\\x22player\\x22\\x3A\\x22Eddie\\x20Nketiah\\x22,\\x22h_a\\x22\\x3A\\x22h\\x22,\\x22player_id\\x22\\x3A\\x226482\\x22,\\x22situation\\x22\\x3A\\x22OpenPlay\\x22,\\x22season\\x22\\x3A\\x222020\\x22,\\x22shotType\\x22\\x3A\\x22RightFoot\\x22,\\x22match_id\\x22\\x3A\\x2214755\\x22,\\x22h_team\\x22\\x3A\\x22Arsenal\\x22,\\x22a_team\\x22\\x3A\\x22Everton\\x22,\\x22h_goals\\x22\\x3A\\x220\\x22,\\x22a_goals\\x22\\x3A\\x221\\x22,\\x22date\\x22\\x3A\\x222021\\x2D04\\x2D23\\x2019\\x3A00\\x3A00\\x22,\\x22player_assisted\\x22\\x3A\\x22Thomas\\x20Partey\\x22,\\x22lastAction\\x22\\x3A\\x22Pass\\x22\\x7D,\\x7B\\x22id\\x22\\x3A\\x22416978\\x22,\\x22minute\\x22\\x3A\\x221\\x22,\\x22result\\x22\\x3A\\x22Bloc

In [8]:
#strip unnecessary symbols, to get the json Data
ind_start = strings.index("('")+2
ind_end = strings.index("')")

json_data = strings[ind_start:ind_end]
json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
data = json.loads(json_data)

In [9]:
data

{'h': [{'id': '416977',
   'minute': '1',
   'result': 'BlockedShot',
   'X': '0.785',
   'Y': '0.7180000305175781',
   'xG': '0.022559041157364845',
   'player': 'Eddie Nketiah',
   'h_a': 'h',
   'player_id': '6482',
   'situation': 'OpenPlay',
   'season': '2020',
   'shotType': 'RightFoot',
   'match_id': '14755',
   'h_team': 'Arsenal',
   'a_team': 'Everton',
   'h_goals': '0',
   'a_goals': '1',
   'date': '2021-04-23 19:00:00',
   'player_assisted': 'Thomas Partey',
   'lastAction': 'Pass'},
  {'id': '416978',
   'minute': '1',
   'result': 'BlockedShot',
   'X': '0.8880000305175781',
   'Y': '0.6140000152587891',
   'xG': '0.09224949777126312',
   'player': 'Dani Ceballos',
   'h_a': 'h',
   'player_id': '2446',
   'situation': 'OpenPlay',
   'season': '2020',
   'shotType': 'RightFoot',
   'match_id': '14755',
   'h_team': 'Arsenal',
   'a_team': 'Everton',
   'h_goals': '0',
   'a_goals': '1',
   'date': '2021-04-23 19:00:00',
   'player_assisted': 'Nicolas Pepe',
   'lastAc

In [31]:
### variables we want to convert into our dataframe
x = []
y = []
xg = []
team = []
result = []
# separate data from home and away team
data_home = data['h']
data_away = data['a']


In [24]:
# home_team
#key is (id, minute, result, X, Y...) all of the elements of the json object
for index in range(len(data_home)):
    for key in data_home[index]:  
        if key == 'X':
            x.append(float(data_home[index][key])*120) #converto string to float and fix coordinates(tanto por 1 a 80*120)
        if key == 'Y':
            y.append(float(data_home[index][key])*80)
        if key == 'xG':
            xg.append(float(data_home[index][key]))
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])

In [32]:
#away_team         
for index in range(len(data_away)):
    for key in data_away[index]:  
        if key == 'X':
            x.append(float(data_away[index][key])*120)
        if key == 'Y':
            y.append(float(data_away[index][key])*80)
        if key == 'xG':
            xg.append(float(data_away[index][key]))
        if key == 'a_team':
            team.append(data_away[index][key])  
        if key == 'result':
            result.append(data_away[index][key])

In [33]:
#create the dataframe
col_names = ['x','y','xg','team','result']
df = pd.DataFrame([x,y,xg,team, result],index=col_names)
df = df.T #transpose data, horizontaly to vertical

In [34]:
df

,x,y,xg,team,result
0,112.68,43.44,0.143436,Everton,MissedShots
1,104.04,29.92,0.096807,Everton,SavedShot
2,90.72,34.16,0.0186057,Everton,BlockedShot
3,93.12,55.2,0.0134152,Everton,MissedShots
4,102.24,36.64,0.0663084,Everton,BlockedShot
5,85.44,46.56,0.0404198,Everton,ShotOnPost
6,89.88,24.08,0.043091,Everton,BlockedShot
7,108.6,36.56,0.108483,Everton,BlockedShot


In [22]:
#save dataframe into a csv file
df.to_csv('shotMapEverton23042021.csv', index = False)